In [1]:
import openreview

# 若你需要使用旧版 API
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username='like2248@163.com',
    password='LIke12345.'
)

In [2]:
conference = ["ACL","ICML","ICLR","NeurIPS"]

CF = "ICLR"
year = "2025"

In [3]:
get_venues = lambda client: client.get_group(id='venues').members
venues = get_venues(client)
print(len(venues)) 
venue_id = None
# venues[0] ICML.cc/2018/Workshop
for venue in venues:
    if CF in venue and year in venue and "Conference" in venue:
        print(venue)
        venue_id=venue

2555
ICLR.cc/2025/Conference


In [4]:
def get_submissions(client, venue_id, status='all'):
    # Retrieve the venue group information
    venue_group = client.get_group(venue_id)
    
    # Define the mapping of status to the respective content field
    status_mapping = {
        "all": venue_group.content['submission_name']['value'],
        "accepted": venue_group.id,  # Assuming 'accepted' status doesn't have a direct field
        "under_review": venue_group.content['submission_venue_id']['value'],
        "withdrawn": venue_group.content['withdrawn_venue_id']['value'],
        "desk_rejected": venue_group.content['desk_rejected_venue_id']['value']
    }
    print(status_mapping)
    # Fetch the corresponding submission invitation or venue ID
    if status in status_mapping:
        if status == "all":
            # Return all submissions regardless of their status
            return client.get_all_notes(invitation=f'{venue_id}/-/{status_mapping[status]}')
        
        # For all other statuses, use the content field 'venueid'
        return client.get_all_notes(content={'venueid': status_mapping[status]})
    
    raise ValueError(f"Invalid status: {status}. Valid options are: {list(status_mapping.keys())}")


In [5]:
submissions = get_submissions(client, venue_id, 'accepted')
#print(submissions[0])
print(len(submissions))


{'all': 'Submission', 'accepted': 'ICLR.cc/2025/Conference', 'under_review': 'ICLR.cc/2025/Conference/Submission', 'withdrawn': 'ICLR.cc/2025/Conference/Withdrawn_Submission', 'desk_rejected': 'ICLR.cc/2025/Conference/Desk_Rejected_Submission'}


Getting V2 Notes: 100%|█████████▉| 3700/3704 [00:02<00:00, 1521.76it/s]

3704


In [16]:
#print(submissions[0].content)
for paper in submissions:
    #print(f"Paper ID: {paper.id}") # string
    #print(f"Title: {paper.content['title']['value']}")# string
    #print(f"keywords: {paper.content['keywords']['value']}") # list
    #print(f"Abstract: {paper.content['abstract']['value']}") # string
    print(f"PDF: {paper.content['pdf']['value']}") # list
    print('---')
    break

PDF: /pdf/5258198d4d738e054b9e119c78ec513874dfa67d.pdf
---


In [7]:
import json

# 构造要保存的数据
papers_data = []

for paper in submissions:
    paper_info = {
        "id": paper.id,
        "title": paper.content['title']['value'],
        "keywords": paper.content['keywords']['value'],
        "abstract": paper.content['abstract']['value'],
        "authors": paper.content['authors']['value'],
        "pdf": paper.content['pdf']['value']
    }
    papers_data.append(paper_info)
import os
import json
print(len(papers_data))
filename = f'paper_list/{CF}_{year}.json'
if not os.path.exists(filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(papers_data, f, ensure_ascii=False, indent=4)
else:
    print(f"文件 {filename} 已存在，跳过保存")

3704


In [19]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="sk-823fc37d344844128a4bbd02e690d198", base_url="https://api.deepseek.com")
message=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=message,
    stream=False
)
message.append({"role": response.choices[0].role, "content": "Hello"}) # 'assistant'
print(response.choices[0])
print(response.choices[0].message.content)

UnprocessableEntityError: Failed to deserialize the JSON body into the target type: messages[0][0]: invalid type: map, expected variant identifier at line 1 column 14